## Transfer from 3d CNN

the codes below needs review. Medyo makalat tong part na 'to

In [9]:
# Predict with a sample video

import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import math

def predict_real_time(video_path, model, encoder, img_size=(216, 216), interval=15, window_size=20):
    """
    Predicts actions every `interval` seconds using 16 evenly spaced frames from the past 5 seconds.
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_buffer = []
    frame_idx = 0
    last_pred_time = 0  # Last prediction time in video seconds

    frames_needed = int(interval * fps)  

    print(f"Processing video: {video_path}")
    print(f"Total frames: {total_frames}, FPS: {fps}")
    print(f"Frames per {interval}-second window: {frames_needed}, Sampling {window_size} frames")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize and normalize frame
        frame = cv2.resize(frame, img_size)
        frame = frame / 255.0
        frame_buffer.append(frame)

        # Keep only the last `frames_needed` frames
        if len(frame_buffer) > frames_needed:
            frame_buffer.pop(0)  # Remove the oldest frame

        # Compute current video time
        current_time = frame_idx / fps  # Convert frame index to seconds

        # Predict every `interval` seconds in video time
        if current_time >= last_pred_time + interval:
            if len(frame_buffer) == frames_needed:  # Ensure we have enough frames
                # evenly spaced frames
                sampled_frames = np.array([frame_buffer[math.floor(i * len(frame_buffer) / window_size)] for i in range(window_size)])

                for i, frame in enumerate(sampled_frames):
                    cv2.imwrite(f"debug_frame_{i}.jpg", (frame * 255).astype(np.uint8))
                    
                # Add batch dimension
                input_data = np.expand_dims(sampled_frames, axis=0)
                pred = model.predict(input_data, verbose=0)
                class_idx = np.argmax(pred)
                predicted_label = encoder.inverse_transform([class_idx])[0]

                # Display prediction with timestamp
                minutes = int(current_time // 60)
                seconds = int(current_time % 60)
                print(f"[{minutes:02d}:{seconds:02d}] Predicted Action: {predicted_label}")

                last_pred_time = current_time  # Update last prediction time
        
        frame_idx += 1

    cap.release()

# Load saved model
model = tf.keras.models.load_model("examination_behavior-20F-216IS-16.keras")

# Initialize encoder with class labels
CLASSES = ['Non-suspicious', 'Suspicious']
encoder = LabelEncoder()
encoder.fit(CLASSES)

# Path to long video
video_path = "D:\Recordings\Recordings\Part1.mp4"

# Predict actions every 5 seconds using 16 sampled frames
predict_real_time(video_path, model, encoder, img_size=(216, 216), interval=15, window_size=20)

ValueError: File not found: filepath=examination_behavior-20F-216IS-16.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
# Predict with the webcam

import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import math
import time

def predict_real_time(model, encoder, img_size=(216, 216), interval=15, window_size=20):
    """
    Predicts actions from the webcam every `interval` seconds using evenly spaced frames from the past `interval` seconds.
    """
    cap = cv2.VideoCapture(0)  # Open webcam
    fps = cap.get(cv2.CAP_PROP_FPS) or 30  # Default to 30 FPS if unknown
    frame_buffer = []
    frame_idx = 0
    last_pred_time = 0  # Track time in seconds

    print("Webcam started. Press 'q' to exit.")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize and normalize frame
        frame_resized = cv2.resize(frame, img_size)
        frame_resized = frame_resized / 255.0
        frame_buffer.append(frame_resized)

        # Keep only the last `frames_needed` frames
        frames_needed = int(interval * fps)  # Frames in past `interval` seconds
        if len(frame_buffer) > frames_needed:
            frame_buffer.pop(0)  # Remove oldest frame

        # Compute current video time
        current_time = frame_idx / fps  # Convert frame index to seconds

        # Predict every `interval` seconds
        if current_time >= last_pred_time + interval:
            if len(frame_buffer) >= window_size:
                # evenly spaced frames
                sampled_frames = np.array([frame_buffer[math.floor(i * len(frame_buffer) / window_size)] for i in range(window_size)])

                # Add batch dimension
                input_data = np.expand_dims(sampled_frames, axis=0)
                pred = model.predict(input_data, verbose=0)
                class_idx = np.argmax(pred)
                predicted_label = encoder.inverse_transform([class_idx])[0]

                # Compute formatted timestamp
                minutes = int(current_time // 60)
                seconds = int(current_time % 60)
                print(f"[{minutes:02d}:{seconds:02d}] Predicted Action: {predicted_label}")

                last_pred_time += interval  # Move to the next prediction time

        # Overlay timestamp on frame
        timestamp_text = f"Time: {time.strftime('%H:%M:%S')}"
        cv2.putText(frame, timestamp_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Show webcam feed
        try:
            cv2.imshow("Webcam Feed", frame)
        except cv2.error:
            pass  # Ignore errors if running in a non-GUI environment

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_idx += 1  # Increase frame count

    cap.release()
    cv2.destroyAllWindows()

# Load saved model
model = tf.keras.models.load_model("examination_behavior-20F-216IS-29.keras")

# Initialize encoder with class labels
CLASSES = ['Non-suspicious', 'Suspicious']
encoder = LabelEncoder()
encoder.fit(CLASSES)

# Predict actions using the webcam
predict_real_time(model, encoder, img_size=(216, 216), interval=15, window_size=20)


Webcam started. Press 'q' to exit.
[00:15] Predicted Action: Suspicious


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import time

from tensorflow.keras.models import load_model

# Constants
IMG_SIZE = 216
FRAMES = 20
CHANNELS = 1  # Grayscale
CLASSES = ['Non-suspicious', 'Suspicious']
SEGMENT_DURATION = 15  # seconds
model_path = "examination_behavior-20F-216IS-31-gray.keras"

# Load the model
model = load_model(model_path)

segment_frames = []
segment_start_time = time.time()

# Open webcam
cap = cv2.VideoCapture(0)

def preprocess_frame(frame):
    frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    frame = frame / 255.0
    frame = np.expand_dims(frame, axis=-1)  # Add channel dimension
    return frame

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Preprocess
    processed = preprocess_frame(gray_frame)
    segment_frames.append(processed)

    current_time = time.time()
    if current_time - segment_start_time >= SEGMENT_DURATION:
        total = len(segment_frames)
        if total >= FRAMES:
            indices = np.linspace(0, total - 1, FRAMES).astype(int)
            sampled_frames = np.array([segment_frames[i] for i in indices])
            input_data = np.expand_dims(sampled_frames, axis=0)  # shape: (1, 20, 216, 216, 1)

            # Predict
            preds = model.predict(input_data, verbose=0)
            label_idx = np.argmax(preds)
            label = CLASSES[label_idx]
            confidence = preds[0][label_idx]

            print(f"Prediction: {label} ({confidence:.2f})")

            # Annotate display
            cv2.putText(gray_frame, f"{label} ({confidence:.2f})", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255), 2)

        segment_frames = []
        segment_start_time = current_time

    # Show live grayscale frame
    cv2.imshow("Live Suspicious Behavior Detection (Grayscale)", gray_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Prediction: Non-suspicious (0.51)
Prediction: Non-suspicious (0.51)
Prediction: Non-suspicious (0.50)
Prediction: Non-suspicious (0.50)
Prediction: Suspicious (0.51)
Prediction: Non-suspicious (0.50)
Prediction: Suspicious (0.50)


### Haven't reviewed this one yet

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
import cv2
from tensorflow.keras.utils import to_categorical

# Load saved model
model = tf.keras.models.load_model("goat_action_3dcnn.h5")
IMG_SIZE = 64  # Resize frames to 64x64
FRAMES = 16  # Fixed number of frames per video
CHANNELS = 3  # RGB channels
CLASSES = ['Running', 'Sitting', 'Grazing']

encoder = LabelEncoder()
encoder.fit(CLASSES)

def extract_video_frames(video_path, num_frames, img_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)  # Select evenly spaced frames
    
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        
        if i in frame_indices:
            frame = cv2.resize(frame, (img_size, img_size))
            frames.append(frame)
    
    cap.release()
    
    if len(frames) < num_frames:  # Pad if not enough frames
        frames.extend([frames[-1]] * (num_frames - len(frames)))
    
    return np.array(frames) if len(frames) == num_frames else None

def predict_video(video_path, model, encoder):
    video_data = extract_video_frames(video_path, FRAMES, IMG_SIZE)
    
    if video_data is None:
        print("Error processing video.")
        return
    
    video_data = np.expand_dims(video_data / 255.0, axis=0)
    prediction = model.predict(video_data)
    
    class_idx = np.argmax(prediction)
    predicted_label = encoder.inverse_transform([class_idx])[0]
    print(f"Predicted Action: {predicted_label}")

# Test with a sample video
predict_video(r"Goat\Train\Grazing\002(front).mp4", model, encoder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Predicted Action: Grazing


In [ ]:
import cv2
import numpy as np

def predict_video_segments(video_path, model, encoder, window_size=16, stride=16, img_size=(64, 64)):
    """

    Args:
        video_path (str): Path to the video file.
        model (tf.keras.Model): Trained 3D CNN model.
        encoder (LabelEncoder): Encoder for decoding class labels.
        window_size (int): Number of frames per prediction window.
        stride (int): Number of frames to move between windows.
        img_size (tuple): Resize dimensions for frames.

    Returns:
        list: List of (timestamp, predicted_action) tuples.
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    predictions = []
    frame_buffer = []
    frame_idx = 0

    print(f"Processing video: {video_path}")
    print(f"Total frames: {total_frames}, FPS: {fps}")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize and normalize frame
        frame = cv2.resize(frame, img_size)
        frame = frame / 255.0
        frame_buffer.append(frame)

        # Predict when enough frames are collected
        if len(frame_buffer) == window_size:
            input_data = np.expand_dims(np.array(frame_buffer), axis=0)  # Shape: (1, window_size, H, W, C)
            pred = model.predict(input_data, verbose=0)
            class_idx = np.argmax(pred)
            predicted_label = encoder.inverse_transform([class_idx])[0]

            # Calculate timestamp
            timestamp = frame_idx / fps
            predictions.append((timestamp, predicted_label))

            # Slide window
            frame_buffer = frame_buffer[stride:]

        frame_idx += 1

    cap.release()

    # Display predictions
    print("\nPredicted Actions:")
    for timestamp, action in predictions:
        minutes = int(timestamp // 60)
        seconds = int(timestamp % 60)
        print(f"[{minutes:02d}:{seconds:02d}] Predicted Action: {action}")

    return predictions

# ----------------------------------

import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Load saved model
model = tf.keras.models.load_model("goat_action_3dcnn.h5")

# Initialize encoder with class labels
CLASSES = ['Running', 'Sitting', 'Grazing']
encoder = LabelEncoder()
encoder.fit(CLASSES)

# Path to long video
video_path = r"stitched_output.mp4"

# Predict actions in long video
predict_video_segments(video_path, model, encoder, window_size=16, stride=16, img_size=(64, 64))


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import time

def predict_real_time(video_path, model, encoder, window_size=16, stride=8, img_size=(64, 64), interval=5):
    """
    Predicts actions from a video stream in real-time every `interval` seconds.

    Args:
        video_path (str): Path to the video file.
        model (tf.keras.Model): Trained 3D CNN model.
        encoder (LabelEncoder): Encoder for decoding class labels.
        window_size (int): Number of frames per prediction window.
        stride (int): Number of frames to move between windows.
        img_size (tuple): Resize dimensions for frames.
        interval (int): Time interval (in seconds) for predictions.
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_buffer = []
    frame_idx = 0
    last_pred_time = time.time()
    
    print(f"Processing video: {video_path}")
    print(f"Total frames: {total_frames}, FPS: {fps}")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize and normalize frame
        frame = cv2.resize(frame, img_size)
        frame = frame / 255.0
        frame_buffer.append(frame)
        
        # Predict every `interval` seconds
        if time.time() - last_pred_time >= interval:
            if len(frame_buffer) >= window_size:
                input_data = np.expand_dims(np.array(frame_buffer[-window_size:]), axis=0)  # Last `window_size` frames
                pred = model.predict(input_data, verbose=0)
                class_idx = np.argmax(pred)
                predicted_label = encoder.inverse_transform([class_idx])[0]

                # Display prediction with timestamp
                timestamp = frame_idx / fps
                minutes = int(timestamp // 60)
                seconds = int(timestamp % 60)
                print(f"[{minutes:02d}:{seconds:02d}] Predicted Action: {predicted_label}")

                last_pred_time = time.time()
        
        frame_idx += 1
    
    cap.release()

# Load saved model
model = tf.keras.models.load_model("goat_action_3dcnn.h5")

# Initialize encoder with class labels
CLASSES = ['Running', 'Sitting', 'Grazing']
encoder = LabelEncoder()
encoder.fit(CLASSES)

# Path to long video
video_path = r"stitched_output.mp4"

# Predict actions in real-time every 5 seconds
predict_real_time(video_path, model, encoder, window_size=16, stride=8, img_size=(64, 64), interval=5)

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import math

def predict_real_time(video_path, model, encoder, img_size=(64, 64), interval=5, window_size=16):
    """
    Predicts actions every `interval` seconds using 16 evenly spaced frames from the past 5 seconds.
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_buffer = []
    frame_idx = 0
    last_pred_time = 0  # Last prediction time in video seconds

    frames_needed = int(interval * fps)  # Total frames in a 5-second window

    print(f"Processing video: {video_path}")
    print(f"Total frames: {total_frames}, FPS: {fps}")
    print(f"Frames per {interval}-second window: {frames_needed}, Sampling {window_size} frames")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize and normalize frame
        frame = cv2.resize(frame, img_size)
        frame = frame / 255.0
        frame_buffer.append(frame)

        # Keep only the last `frames_needed` frames
        if len(frame_buffer) > frames_needed:
            frame_buffer.pop(0)  # Remove the oldest frame

        # Compute current video time
        current_time = frame_idx / fps  # Convert frame index to seconds

        # Predict every `interval` seconds in video time
        if current_time >= last_pred_time + interval:
            if len(frame_buffer) == frames_needed:  # Ensure we have enough frames
                # evenly spaced frames
                sampled_frames = np.array([frame_buffer[math.floor(i * len(frame_buffer) / window_size)] for i in range(window_size)])

                # Add batch dimension
                input_data = np.expand_dims(sampled_frames, axis=0)
                pred = model.predict(input_data, verbose=0)
                class_idx = np.argmax(pred)
                predicted_label = encoder.inverse_transform([class_idx])[0]

                # Display prediction with timestamp
                minutes = int(current_time // 60)
                seconds = int(current_time % 60)
                print(f"[{minutes:02d}:{seconds:02d}] Predicted Action: {predicted_label}")

                last_pred_time = current_time  # Update last prediction time
        
        frame_idx += 1

    cap.release()

# Load saved model
model = tf.keras.models.load_model("goat_action_3dcnn.h5")

# Initialize encoder with class labels
CLASSES = ['Running', 'Sitting', 'Grazing']
encoder = LabelEncoder()
encoder.fit(CLASSES)

# Path to long video
video_path = r"stitched_output.mp4"

# Predict actions every 5 seconds using 16 sampled frames
predict_real_time(video_path, model, encoder, img_size=(64, 64), interval=5, window_size=16)


Processing video: stitched_output.mp4
Total frames: 4540, FPS: 41.91179785039039
Frames per 5-second window: 209, Sampling 16 frames
[00:05] Predicted Action: Grazing
[00:10] Predicted Action: Grazing
[00:15] Predicted Action: Grazing
[00:20] Predicted Action: Grazing
[00:25] Predicted Action: Grazing
[00:30] Predicted Action: Running
[00:35] Predicted Action: Grazing
[00:40] Predicted Action: Grazing
[00:45] Predicted Action: Grazing
[00:50] Predicted Action: Grazing
[00:55] Predicted Action: Grazing
[01:00] Predicted Action: Running
[01:05] Predicted Action: Sitting
[01:10] Predicted Action: Sitting
[01:15] Predicted Action: Sitting
[01:20] Predicted Action: Grazing
[01:25] Predicted Action: Grazing
[01:30] Predicted Action: Grazing
[01:35] Predicted Action: Grazing
[01:40] Predicted Action: Grazing
[01:45] Predicted Action: Grazing


In [ ]:
# Initialize GUI - Di ko pa alam kung paano to gagawin. HAHAHAH

import cv2
import numpy as np
import tensorflow as tf
import math
import time
import tkinter as tk
import customtkinter as ctk
from PIL import Image, ImageTk
from sklearn.preprocessing import LabelEncoder

class GoatActionApp:
    def __init__(self, root, model, encoder, img_size=(216, 216), interval=15, window_size=20):
        self.root = root
        self.root.title("Goat Action Recognition")
        self.model = model
        self.encoder = encoder
        self.img_size = img_size
        self.interval = interval
        self.window_size = window_size
        self.frame_buffer = []
        self.cap = cv2.VideoCapture(0)
        self.fps = self.cap.get(cv2.CAP_PROP_FPS) or 30
        self.frame_idx = 0
        self.last_pred_time = 0
        self.predicted_label = "Waiting..."
        self.image_tk = None  # Store reference to prevent garbage collection
        
        # Create GUI layout
        self.root.geometry("800x600")
        self.root.resizable(False, False)
        
        # Webcam frame
        self.video_frame = ctk.CTkLabel(self.root, text="", width=600, height=450)
        self.video_frame.place(relx=0.5, rely=0.4, anchor="center")
        
        # Prediction label
        self.prediction_label = ctk.CTkLabel(self.root, text=self.predicted_label, font=("Arial", 20))
        self.prediction_label.place(relx=0.5, rely=0.85, anchor="center")
        
        self.update_video()

    def update_video(self):
        ret, frame = self.cap.read()
        if not ret:
            self.root.after(10, self.update_video)
            return
        
        # Resize for prediction
        frame_resized = cv2.resize(frame, self.img_size) / 255.0
        self.frame_buffer.append(frame_resized)
        
        # Keep only the last required frames
        frames_needed = int(self.interval * self.fps)
        if len(self.frame_buffer) > frames_needed:
            self.frame_buffer.pop(0)
        
        # Compute current time
        current_time = self.frame_idx / self.fps
        
        if current_time >= self.last_pred_time + self.interval:
            if len(self.frame_buffer) >= self.window_size:
                sampled_frames = np.array([
                    self.frame_buffer[math.floor(i * len(self.frame_buffer) / self.window_size)]
                    for i in range(self.window_size)
                ])
                input_data = np.expand_dims(sampled_frames, axis=0)
                pred = self.model.predict(input_data, verbose=0)
                class_idx = np.argmax(pred)
                self.predicted_label = self.encoder.inverse_transform([class_idx])[0]
                self.prediction_label.configure(text=f"Predicted Action: {self.predicted_label}")
                self.last_pred_time += self.interval
        
        # Convert to Tkinter format
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (600, 450))
        self.image_tk = ImageTk.PhotoImage(Image.fromarray(frame))  # Store reference
        self.video_frame.configure(image=self.image_tk)
        self.video_frame.image = self.image_tk
        
        self.frame_idx += 1
        self.root.after(10, self.update_video)
    
    def run(self):
        self.root.mainloop()
        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    model = tf.keras.models.load_model("examination_behavior-20F-216IS-16.keras")
    CLASSES = ['Suspicious', 'Non-suspicious']
    encoder = LabelEncoder()
    encoder.fit(CLASSES)
    root = ctk.CTk()
    app = GoatActionApp(root, model, encoder)
    app.run()


c:\Users\Raymond\Desktop\3D-CNN\venv\lib\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")
c:\Users\Raymond\Desktop\3D-CNN\venv\lib\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")
c:\Users\Raymond\Desktop\3D-CNN\venv\lib\site-packages\customtkinter\windows\wid

In [ ]:
# Initialize GUI - Di ko pa alam kung paano to gagawin. HAHAHAH

import cv2
import numpy as np
import tensorflow as tf
import math
import tkinter as tk
import customtkinter as ctk
from PIL import Image, ImageTk
from sklearn.preprocessing import LabelEncoder

class GoatActionApp:
    def __init__(self, root, model, encoder, img_size=(216, 216), interval=15, window_size=20):
        self.root = root
        self.root.title("Goat Action Recognition")
        self.model = model
        self.encoder = encoder
        self.img_size = img_size
        self.interval = interval
        self.window_size = window_size
        self.frame_buffer = []
        self.cap = cv2.VideoCapture(0)
        self.fps = self.cap.get(cv2.CAP_PROP_FPS) or 30
        self.frame_idx = 0
        self.last_pred_time = 0
        self.predicted_label = "Waiting..."
        self.image_tk = None
        self.seconds_elapsed = 1  # Start from 1
        
        # GUI Layout
        self.root.geometry("800x600")
        self.root.resizable(False, False)
        
        self.video_frame = ctk.CTkLabel(self.root, text="", width=600, height=450)
        self.video_frame.place(relx=0.5, rely=0.4, anchor="center")
        
        self.prediction_label = ctk.CTkLabel(self.root, text=self.predicted_label, font=("Arial", 20))
        self.prediction_label.place(relx=0.5, rely=0.85, anchor="center")

        self.timer_label = ctk.CTkLabel(self.root, text=f"Time since last prediction: {self.seconds_elapsed}s", font=("Arial", 16))
        self.timer_label.place(relx=0.5, rely=0.92, anchor="center")
        
        self.update_video()
        self.update_timer()

    def update_video(self):
        ret, frame = self.cap.read()
        if not ret:
            self.root.after(10, self.update_video)
            return
        
        frame_resized = cv2.resize(frame, self.img_size) / 255.0
        self.frame_buffer.append(frame_resized)
        
        frames_needed = int(self.interval * self.fps)
        if len(self.frame_buffer) > frames_needed:
            self.frame_buffer.pop(0)
        
        current_time = self.frame_idx / self.fps

        if current_time >= self.last_pred_time + self.interval:
            if len(self.frame_buffer) >= self.window_size:
                sampled_frames = np.array([
                    self.frame_buffer[math.floor(i * len(self.frame_buffer) / self.window_size)]
                    for i in range(self.window_size)
                ])
                input_data = np.expand_dims(sampled_frames, axis=0)
                pred = self.model.predict(input_data, verbose=0)
                class_idx = np.argmax(pred)
                self.predicted_label = self.encoder.inverse_transform([class_idx])[0]
                self.prediction_label.configure(text=f"Predicted Action: {self.predicted_label}")
                self.last_pred_time += self.interval
                self.seconds_elapsed = 1  # Reset to 1

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (600, 450))
        self.image_tk = ImageTk.PhotoImage(Image.fromarray(frame))
        self.video_frame.configure(image=self.image_tk)
        self.video_frame.image = self.image_tk
        
        self.frame_idx += 1
        self.root.after(10, self.update_video)
    
    def update_timer(self):
        if self.seconds_elapsed < self.interval:
            self.seconds_elapsed += 1
        self.timer_label.configure(text=f"Time since last prediction: {self.seconds_elapsed}s")
        self.root.after(1000, self.update_timer)

    def run(self):
        self.root.mainloop()
        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    model = tf.keras.models.load_model("examination_behavior-20F-216IS-16.keras")
    CLASSES = ['Suspicious', 'Non-suspicious']
    encoder = LabelEncoder()
    encoder.fit(CLASSES)
    root = ctk.CTk()
    app = GoatActionApp(root, model, encoder)
    app.run()


c:\Users\Raymond\Desktop\3D-CNN\venv\lib\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")
c:\Users\Raymond\Desktop\3D-CNN\venv\lib\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")
c:\Users\Raymond\Desktop\3D-CNN\venv\lib\site-packages\customtkinter\windows\wid

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import math
import tkinter as tk
import customtkinter as ctk
from PIL import Image, ImageTk
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os
from datetime import datetime

class BehaviorMonitoringApp:
    def __init__(self, root, model, encoder, img_size=(216, 216), interval=15, window_size=20):
        self.root = root
        self.root.title("Student Behavior Monitoring")
        self.model = model
        self.encoder = encoder
        self.img_size = img_size
        self.interval = interval
        self.window_size = window_size
        self.frame_buffer = []
        self.cap = cv2.VideoCapture(0)
        self.fps = self.cap.get(cv2.CAP_PROP_FPS) or 30
        self.frame_idx = 0
        self.last_pred_time = 0
        self.predicted_label = "Waiting..."
        self.image_tk = None
        self.seconds_elapsed = 1

        # Initialize CSV saving
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        self.save_folder = "monitoring_logs"
        os.makedirs(self.save_folder, exist_ok=True)
        self.monitor_csv_path = os.path.join(self.save_folder, f"monitoring_log_{timestamp}.csv")
        self.landmark_csv_path = os.path.join(self.save_folder, f"landmarks_log_{timestamp}.csv")
        self.logs = []
        self.landmark_logs = []

        # GUI Layout
        self.root.geometry("800x600")
        self.root.resizable(False, False)
        
        self.video_frame = ctk.CTkLabel(self.root, text="", width=600, height=450)
        self.video_frame.place(relx=0.5, rely=0.4, anchor="center")
        
        self.prediction_label = ctk.CTkLabel(self.root, text=self.predicted_label, font=("Arial", 20))
        self.prediction_label.place(relx=0.5, rely=0.85, anchor="center")

        self.timer_label = ctk.CTkLabel(self.root, text=f"Time since last prediction: {self.seconds_elapsed}s", font=("Arial", 16))
        self.timer_label.place(relx=0.5, rely=0.92, anchor="center")
        
        self.update_video()
        self.update_timer()

    def update_video(self):
        ret, frame = self.cap.read()
        if not ret:
            self.root.after(10, self.update_video)
            return
        
        frame_resized = cv2.resize(frame, self.img_size) / 255.0
        self.frame_buffer.append(frame_resized)
        
        frames_needed = int(self.interval * self.fps)
        if len(self.frame_buffer) > frames_needed:
            self.frame_buffer.pop(0)
        
        current_time = self.frame_idx / self.fps

        if current_time >= self.last_pred_time + self.interval:
            if len(self.frame_buffer) >= self.window_size:
                sampled_frames = np.array([
                    self.frame_buffer[math.floor(i * len(self.frame_buffer) / self.window_size)]
                    for i in range(self.window_size)
                ])
                input_data = np.expand_dims(sampled_frames, axis=0)  # (1, 20, 216, 216, 3)
                
                # Multitask model output: 4 heads
                classification_pred, landmarks_pred, motion_pred, velocity_pred = self.model.predict(input_data, verbose=0)

                # --- CLASSIFICATION ---
                class_idx = np.argmax(classification_pred[0])
                self.predicted_label = self.encoder.inverse_transform([class_idx])[0]
                self.prediction_label.configure(text=f"Predicted: {self.predicted_label}")

                # --- SAVE LANDMARKS for 20 FRAMES CLEANLY (X, Y separate) ---
                landmarks = landmarks_pred[0]  # (142,) → (71 points × 2 coordinates)
                landmarks = landmarks.reshape(71, 2)  # (71, 2)

                time_now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                for idx in range(self.window_size):
                    frame_landmark_entry = {
                        "Time": time_now,
                        "Frame_Index": idx
                    }
                    for point_idx, (x, y) in enumerate(landmarks):
                        frame_landmark_entry[f"Landmark_{point_idx}_X"] = x
                        frame_landmark_entry[f"Landmark_{point_idx}_Y"] = y
                    self.landmark_logs.append(frame_landmark_entry)

                # --- Save Monitoring Log (Motion, Velocity) ---
                motion = motion_pred[0]
                velocity = velocity_pred[0]
                log_entry = {
                    "Time": time_now,
                    "Prediction": self.predicted_label
                }
                for i in range(len(motion)):
                    log_entry[f"Motion_{i}"] = motion[i]
                    log_entry[f"Velocity_{i}"] = velocity[i]
                self.logs.append(log_entry)

                self.last_pred_time += self.interval
                self.seconds_elapsed = 1

        # Update GUI
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (600, 450))
        self.image_tk = ImageTk.PhotoImage(Image.fromarray(frame))
        self.video_frame.configure(image=self.image_tk)
        self.video_frame.image = self.image_tk
        
        self.frame_idx += 1
        self.root.after(10, self.update_video)
    
    def update_timer(self):
        if self.seconds_elapsed < self.interval:
            self.seconds_elapsed += 1
        self.timer_label.configure(text=f"Time since last prediction: {self.seconds_elapsed}s")
        self.root.after(1000, self.update_timer)

    def run(self):
        self.root.mainloop()
        self.cap.release()
        cv2.destroyAllWindows()

        # Save logs to CSVs
        if self.logs:
            df_monitor = pd.DataFrame(self.logs)
            df_monitor.to_csv(self.monitor_csv_path, index=False)
            print(f"Monitoring logs saved to: {self.monitor_csv_path}")

        if self.landmark_logs:
            df_landmarks = pd.DataFrame(self.landmark_logs)
            df_landmarks.to_csv(self.landmark_csv_path, index=False)
            print(f"Landmark logs saved to: {self.landmark_csv_path}")

if __name__ == "__main__":
    model = tf.keras.models.load_model("models/examination_behavior-20F-216IS-v1.05.keras")
    CLASSES = ['Non-suspicious', 'Suspicious']
    encoder = LabelEncoder()
    encoder.fit(CLASSES)
    root = ctk.CTk()
    app = BehaviorMonitoringApp(root, model, encoder)
    app.run()


c:\Users\Raymond\Desktop\3D-CNN\venv\lib\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")
c:\Users\Raymond\Desktop\3D-CNN\venv\lib\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")
c:\Users\Raymond\Desktop\3D-CNN\venv\lib\site-packages\customtkinter\windows\wid

Monitoring logs saved to: monitoring_logs\monitoring_log_20250428-153629.csv
Landmark logs saved to: monitoring_logs\landmarks_log_20250428-153629.csv


### Previous

In [ ]:
#load with memory

import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Constants
IMG_SIZE = 216  # Resize frames to 64x64
FRAMES = 20  # Fixed number of frames per video
CHANNELS = 3  # RGB channels
CLASSES = ['Suspicious', 'Non-suspicious']

# Function to load videos and convert them to arrays
def load_videos_from_folder(folder_path, class_labels):
    data, labels = [], []
    
    for class_name in class_labels:
        class_path = os.path.join(folder_path, class_name)
        
        for video_name in os.listdir(class_path):
            video_path = os.path.join(class_path, video_name)
            frames = extract_video_frames(video_path, FRAMES, IMG_SIZE)
            
            if frames is not None:
                data.append(frames)
                labels.append(class_name)
    
    return np.array(data), np.array(labels)

# Function to extract fixed number of frames from a video
def extract_video_frames(video_path, num_frames, img_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)  # Select evenly spaced frames
    
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        
        if i in frame_indices:
            frame = cv2.resize(frame, (img_size, img_size))
            frames.append(frame)
    
    cap.release()
    
    if len(frames) < num_frames:  # Pad if not enough frames
        frames.extend([frames[-1]] * (num_frames - len(frames)))
    
    return np.array(frames) if len(frames) == num_frames else None

# Load datasets
test_videos, test_labels = load_videos_from_folder("dataset/test", CLASSES)

# Normalize data
test_videos = test_videos / 255.0

# Encode labels
encoder = LabelEncoder()
encoder.fit(test_labels)  # Fit before transform
test_labels_enc = to_categorical(encoder.transform(test_labels))


print(f"Train shape: {test_videos.shape}, Train labels: {test_labels_enc.shape}")

Train shape: (36, 20, 216, 216, 3), Train labels: (36, 2)


In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model("examination_behavior-20F-216IS-16.keras")

In [ ]:
CLASSES = ['Non-suspicious', 'Suspicious']


def predict_video(video_path, model, encoder):
    video_data = extract_video_frames(video_path, FRAMES, IMG_SIZE)
    
    if video_data is None:
        print("Error processing video.")
        return
    
    video_data = np.expand_dims(video_data / 255.0, axis=0)
    prediction = model.predict(video_data)
    
    class_idx = np.argmax(prediction)
    predicted_label = encoder.inverse_transform([class_idx])[0]
    print(f"Predicted Action: {predicted_label}")

# Test with a sample video
predict_video(r"dataset\test\Non-suspicious\Video 04-082.mp4", model, encoder)
predict_video(r"dataset\test\Suspicious\Video 04-037.mp4", model, encoder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
Predicted Action: Non-suspicious
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
Predicted Action: Suspicious


In [ ]:
def predict_videos_in_folder(folder_path, model, encoder):
    if model is None or encoder is None:
        print("Model or encoder not provided.")
        return

    for class_folder in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_folder)
        if not os.path.isdir(class_path):
            continue

        for video_name in os.listdir(class_path):
            video_path = os.path.join(class_path, video_name)
            if not video_name.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
                continue

            video_data = extract_video_frames(video_path, FRAMES, IMG_SIZE)
            if video_data is None:
                print(f"[SKIPPED] {video_name}: error extracting frames.")
                continue

            video_data = np.expand_dims(video_data / 255.0, axis=0)
            prediction = model.predict(video_data)
            class_idx = np.argmax(prediction)
            predicted_label = encoder.inverse_transform([class_idx])[0]

            print(f"{video_name} => Predicted: {predicted_label}")


In [ ]:
predict_videos_in_folder("dataset/test", model, encoder)